In [ ]:
# Before starting the implementation try understanding this article
# https://www.linkedin.com/pulse/understanding-batch-normalization-layer-group-implementing-pasha-s/

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("Dataset-SA.csv")
df.head()

,product_name,product_price,Rate,Review,Summary,Sentiment
0,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,super!,great cooler excellent air flow and for this p...,positive
1,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,awesome,best budget 2 fit cooler nice cooling,positive
2,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,the quality is good but the power of air is de...,positive
3,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,1,useless product,very bad product its a only a fan,negative
4,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,ok ok product,neutral


In [4]:
df = df[['Review','Sentiment']]
df = df.dropna()
df.shape

(180388, 2)

In [5]:
df['Review'] = [str(text) for text in df['Review']]

In [6]:
from sklearn.preprocessing import LabelEncoder
# Create an instance of LabelEncoder
label_encoder = LabelEncoder()
# Fit the label encoder on the data and transform the data
encoded_data = label_encoder.fit_transform(df['Sentiment'])
df['Sentiment_Coded'] = encoded_data

In [7]:
# Create Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=1000,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True,
    split=' ',
    char_level=False,
    #oov_token="<OOV>",
    analyzer=None,
    )
tokenizer.fit_on_texts(df['Review'])

In [8]:
max_sequence_length = max([len(i.split()) for i in df['Review']])
max_sequence_length

22

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df['Review'], df['Sentiment_Coded'], test_size=0.2, random_state=42, stratify=df['Sentiment_Coded'])

In [10]:
x_train, x_test, y_train, y_test  = list(x_train), list(x_test), list(y_train), list(y_test)

In [11]:
sequences_train = tokenizer.texts_to_sequences(x_train)
sequences_test =  tokenizer.texts_to_sequences(x_test)

In [12]:
# Pad Sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
pad_sequence_train = pad_sequences(sequences_train, maxlen=max_sequence_length)
pad_sequence_test = pad_sequences(sequences_test, maxlen=max_sequence_length)

In [13]:
vocab_size = len(tokenizer.index_word)
output_size = len(df['Sentiment'].unique())
vocab_size, output_size

(1320, 3)

In [14]:
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, LSTM, Bidirectional, Dropout, LayerNormalization
model = Sequential()
model.add(Input(shape=(max_sequence_length,)))
model.add(Embedding(input_dim=vocab_size+1, output_dim=128, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(units=128, return_sequences=True)))
model.add(LayerNormalization())
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(units=128)))
model.add(LayerNormalization())
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(LayerNormalization())
model.add(Dense(32,activation='relu'))
model.add(LayerNormalization())
model.add(Dense(output_size,activation='sigmoid'))
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 22, 128)           169088    
                                                                 
 bidirectional_2 (Bidirecti  (None, 22, 256)           263168    
 onal)                                                           
                                                                 
 layer_normalization_1 (Lay  (None, 22, 256)           512       
 erNormalization)                                                
                                                                 
 dropout_2 (Dropout)         (None, 22, 256)           0         
                                                                 
 bidirectional_3 (Bidirecti  (None, 256)               394240    
 onal)                                                           
                                                      

In [18]:
model.fit(pad_sequence_train, y_train_categorical, epochs = 1, batch_size = 1028, validation_data=(pad_sequence_test, y_test_categorical))

564/564 [==============================] - 325s 560ms/step - loss: 0.3356 - accuracy: 0.9030 - val_loss: 0.3081 - val_accuracy: 0.9119


In [25]:
batch_norm_layer  = model.layers[2]
weights_gamma, weights_beta = batch_norm_layer.get_weights()
len(weights_gamma), len(weights_beta), weights_gamma, weights_beta

(256,
 256,
 array([0.9830796 , 1.0040729 , 1.0052541 , 0.9913805 , 0.9893744 ,
        1.0025591 , 1.0030426 , 0.99195087, 0.9861785 , 0.9868835 ,
        1.0084875 , 0.99918526, 0.9638314 , 0.9730602 , 0.9921283 ,
        1.0072767 , 0.99206436, 0.9980665 , 0.9926398 , 0.9994466 ,
        1.001831  , 1.0024793 , 0.98435956, 1.0052801 , 0.99920976,
        1.005541  , 0.99045616, 0.97551155, 0.9957907 , 0.9920549 ,
        0.99018294, 0.9830635 , 0.99225795, 0.9745829 , 0.98539305,
        0.99470687, 1.0173415 , 0.9943578 , 1.0014411 , 1.0066648 ,
        1.0070728 , 0.9875877 , 1.004114  , 1.0267446 , 0.99372685,
        0.99419457, 0.9811692 , 1.0055975 , 0.99640775, 0.99704933,
        0.99725467, 1.0042762 , 0.9976349 , 1.0143614 , 1.0078384 ,
        0.9971726 , 0.985462  , 0.9896487 , 0.9964412 , 0.9838067 ,
        1.0044657 , 0.9863792 , 0.993771  , 0.9879603 , 1.0072818 ,
        1.0006331 , 0.9744721 , 1.00126   , 0.9967638 , 0.99414104,
        0.99642193, 0.9999506 , 0.99

In [22]:
text= x_test[0]
sequences_inference = tokenizer.texts_to_sequences([text])
padded_sequences_inference = pad_sequences(sequences_inference,maxlen=max_sequence_length)

In [23]:
sequences_inference

[[32, 33, 6]]

In [24]:
model.predict(padded_sequences_inference)

1/1 [==============================] - 2s 2s/step


array([[0.13517539, 0.21910079, 0.9238637 ]], dtype=float32)

In [25]:
model.predict(padded_sequences_inference[0].reshape(1,22))

1/1 [==============================] - 0s 31ms/step


array([[0.13517539, 0.21910079, 0.9238637 ]], dtype=float32)